# Exercise 2: Bumeran Webscrapping

In [81]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
import pandas as pd

### 1. Extraer Links

In [ ]:
def scrappearPagina(driver, urlBase, pagina, ANUNCIOS):
    '''
    funcion que extrae los links de una pagina
    '''

    wait = WebDriverWait(driver, 10)
    driver.get(urlBase + "?page=" + str(pagina))
    time.sleep(3)

    wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, "//a[contains(@href, '/empleos/')]")
    ))

    # Extraer y filtrar enlaces
    enlaces = driver.find_elements(By.XPATH, "//a[contains(@href, '/empleos/')]")
    urls = [e.get_attribute("href") for e in enlaces if e.get_attribute("href")]
    urls_filtradas = list(set([u for u in urls if "bumeran.com.pe/empleos/" in u]))
    
    ANUNCIOS.update(urls_filtradas)

In [ ]:
urlBase = 'https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html'
ANUNCIOS = set()

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.maximize_window() 

for pagina in range(1, 7):    
    print("Procesando Página" + " " + str(pagina))
    scrappearPagina(driver, urlBase, pagina, ANUNCIOS)

#driver.quit()


Procesando Página 1
Procesando Página 2
Procesando Página 3
Procesando Página 4
Procesando Página 5
Procesando Página 6


In [56]:
# 10 links de anuncios
list(ANUNCIOS)[:10]

['https://www.bumeran.com.pe/empleos/desarrollador-full-stack-senior--ecommerce-magento-perfumerias-unidas-1117965886.html',
 'https://www.bumeran.com.pe/empleos/analista-de-programacion-rpg-izipay-1117080379.html',
 'https://www.bumeran.com.pe/empleos/desarrollador-full-stack-surco-presencial-well-consulting-services-1117978454.html',
 'https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html',
 'https://www.bumeran.com.pe/empleos/desarrolladora-mineria-lurin-zamine-peru-1117976772.html',
 'https://www.bumeran.com.pe/empleos/soporte-jr-de-aplicaciones-linux-plsql-hitss-peru-1117963093.html',
 'https://www.bumeran.com.pe/empleos/desarrollador-frontend-senior-fractal-soluciones-it-1117974205.html',
 'https://www.bumeran.com.pe/empleos/programador-ologgi-s.a.c-1117973857.html',
 'https://www.bumeran.com.pe/empleos/analista-de-imagenes-hospitalarias-sistemas-grupo-san-pablo-1117977289.html',
 'https://www.bumeran.com.pe/empleos/analista-programador-sede-ate-bumeran-sel

### 2. Extraer Información

In [76]:
def sracapearAnuncio(url, job_data, driver, te):
    '''
    extrae informacion de un anuncio
    '''

    driver.get(url)
    time.sleep(te)
    
    try:
        job_title = driver.find_element(By.CSS_SELECTOR, "h1").text.strip()
    except NoSuchElementException:
        job_title = None

    try:
        description = driver.find_element(
            By.XPATH, "//h3[contains(text(),'Descripción del puesto')]/following::p[1]"
        ).text.strip()
        if "Beneficios" in description:
            description = description.split("Beneficios")[0].strip()
    except NoSuchElementException:
        description = None

    try:
        district = driver.find_element(By.XPATH, "//a[h2]").text.strip()
        district = district.split(",")[0].strip()
    except NoSuchElementException:
        district = None

    try:
        work_mode = driver.find_element(
            By.XPATH, "//a[contains(@href,'modalidad')]/p"
        ).text.strip()
    except NoSuchElementException:
        work_mode = None

    job_data.append({
        "Job Title": job_title,
        "Description": description,
        "District": district,
        "Work Mode": work_mode,
    })

In [79]:
'''
Estimados:

    Si pongo un time.sleep muy alto me aseguraría que nunca haya errores,
    pero no es muy eficiente, ¿no?

    En esos casos suele ser útil un arreglo de este tipo: el código
    intenta con un time.sleep bajo y, si except, intenta nuevamente
    con un time.sleep mayor, hasta un máximo de 5 intentos.

    Esta ha sido la solución en algunos de mis proyectos, espero que les
    sea útil.

César A. Bueno B.
'''

anunciosDATA = []
maxInt = 5

for i, url in enumerate(ANUNCIOS, 1):
    intento = 1
    print(f"[{i}/{len(ANUNCIOS)}] Accediendo: {url}")

    while intento <= maxInt:
        try:
            te = 2.5*intento-1.5
            resultado = sracapearAnuncio(url, anunciosDATA, driver, te)
            break
        except Exception as e:
            print(f"⚠️ Error: {e} — intentando de nuevo...")
            intento += 1

[1/106] Accediendo: https://www.bumeran.com.pe/empleos/desarrollador-full-stack-senior--ecommerce-magento-perfumerias-unidas-1117965886.html
[2/106] Accediendo: https://www.bumeran.com.pe/empleos/analista-de-programacion-rpg-izipay-1117080379.html
[3/106] Accediendo: https://www.bumeran.com.pe/empleos/desarrollador-full-stack-surco-presencial-well-consulting-services-1117978454.html
[4/106] Accediendo: https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html
[5/106] Accediendo: https://www.bumeran.com.pe/empleos/desarrolladora-mineria-lurin-zamine-peru-1117976772.html
[6/106] Accediendo: https://www.bumeran.com.pe/empleos/soporte-jr-de-aplicaciones-linux-plsql-hitss-peru-1117963093.html
[7/106] Accediendo: https://www.bumeran.com.pe/empleos/desarrollador-frontend-senior-fractal-soluciones-it-1117974205.html
[8/106] Accediendo: https://www.bumeran.com.pe/empleos/programador-ologgi-s.a.c-1117973857.html
[9/106] Accediendo: https://www.bumeran.com.pe/empleos/analista-d

In [83]:
df = pd.DataFrame(anunciosDATA)
df.head()

,Job Title,Description,District,Work Mode
0,Desarrollador Full Stack Senior – Ecommerce (M...,Buscamos un Desarrollador Full Stack Senior co...,Magdalena del Mar,Híbrido
1,Analista de Programación RPG,¡Somos Izipay! La red multimarca más grande de...,Miraflores,Híbrido
2,Desarrollador Full Stack / Surco / Presencial,"En Well Human Resources, estamos en la búsqued...",Santiago de Surco,Presencial
3,Analista Programador Java,Analista Programador Java\nResumen del Puesto\...,Lima,Presencial
4,Desarrollador(a) - Minería - Lurín,Somos una empresa subsidiaria de la Corporació...,Lurín,Presencial


In [90]:
df.to_csv("../out/bumeran_job_details.csv", index=False, encoding="utf-8")